In [1]:
# paper Neural Architectures for Named Entity Recognition 
#ref   https://zhuanlan.zhihu.com/p/79552594
#ref   https://zhuanlan.zhihu.com/p/44042528

#for train           LOSS = -log(     P_realpath / (P_1+P_2+P_3+...)        )
#                         = -log(     e^S_realpath / (e^S_1+e^S_2+e^S_3+..)    )
#                         =  log(e^S_1  + e^S_2 + e^S_3 +...) - S_realpath
#                         =  forward_score - gold_scorehttps://arxiv.org/abs/1603.01360

In [2]:
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.optim as optim

torch.manual_seed(1)

In [3]:
def argmax(vec):
    # return the argmax as a python int
    _, idx = torch.max(vec, 1)
    return idx.item()


def prepare_sequence(seq, to_ix):
    idxs = [to_ix[w] for w in seq]
    return torch.tensor(idxs, dtype=torch.long)


# Compute log sum exp in a numerically stable way for the forward algorithm
def log_sum_exp(vec): #vec=(1,out)=(1,5)=tensor([[-1.0000e+04, -1.0002e+04, -1.0001e+04, -1.3660e+00, -2.0000e+04]])
    #    return= log(e^vec[0}+e^vec[1]+e^vec[2]+e^vec[3]+e^vec[4])
    #先做减法的原因在于，减去最大值可以避免e的指数次导致计算机上溢
    max_score = vec[0, argmax(vec)]  #tensor(-1.3660)
    max_score_broadcast = max_score.view(1, -1).expand(1, vec.size()[1]) #tensor([[-1.3660, -1.3660, -1.3660, -1.3660, -1.3660]],)
    return max_score +  torch.log(torch.sum(torch.exp(vec - max_score_broadcast)))             #tensor([-1.3660], grad_fn=<ViewBackward>)


In [4]:

class BiLSTM_CRF(nn.Module):
#  model = BiLSTM_CRF(len(word_to_ix), tag_to_ix, EMBEDDING_DIM, HIDDEN_DIM)
    def __init__(self, vocab_size, tag_to_ix, embedding_dim, hidden_dim):
        super(BiLSTM_CRF, self).__init__()
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.vocab_size = vocab_size
        self.tag_to_ix = tag_to_ix
        self.tagset_size = len(tag_to_ix)

        self.word_embeds = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim // 2,
                            num_layers=1, bidirectional=True)

        # Maps the output of the LSTM into tag space.
        self.hidden2tag = nn.Linear(hidden_dim, self.tagset_size)

        # Matrix of transition parameters. 转移矩阵是随机的，而且放入了网络中，是会更新的
        self.transitions = nn.Parameter( torch.randn(self.tagset_size, self.tagset_size))  #(out,out)=(5,5)

        # These two statements enforce the constraint that we never transfer to the start tag and we never transfer from the stop tag
        #建立转移矩阵A，并加了两个我们不会变动的约束条件：1是我们不会从其他tag转向start。2是不会从stop开始转向其他。所以这些位置设为-1e4
        #即类似于将矩阵中start那一行及stop那一列添加了约束
        self.transitions.data[tag_to_ix[START_TAG], :] = -10000
        self.transitions.data[:, tag_to_ix[STOP_TAG]] = -10000

        self.hidden = self.init_hidden()

    def init_hidden(self):
        return (torch.randn(2, 1, self.hidden_dim // 2),
                torch.randn(2, 1, self.hidden_dim // 2))


    # 预测序列的得分      return  forward_score = log(e^S_1  + e^S_2 + e^S_3 +...)
    def _forward_alg(self, feats):    #feats=(11,5)=(m,out)
        # Do the forward algorithm to compute the partition function
        init_alphas = torch.full((1, self.tagset_size), -10000.) #(1,5)=tensor([[-10000.], [-10000.], [-10000.], [-10000.], [-10000.]])
        # START_TAG has all of the score.
        init_alphas[0][self.tag_to_ix[START_TAG]] = 0.          #tensor([[-10000., -10000., -10000.,      0., -10000.]])

        # Wrap in a variable so that we will get automatic backprop  类似于第一个单词
        forward_var = init_alphas

        # Iterate through the sentence
        for feat in feats:      #feat=(5) =tensor([-0.2473,  0.2108, -0.3820,  0.4365, -0.3686], grad_fn=<SelectBackward>)
            #一个feat迭代表示一次单词位置转移
            alphas_t = []  # The forward tensors at this timestep
            for next_tag in range(self.tagset_size):
                # broadcast the emission score: it is the same regardless of the previous tag
                emit_score = feat[next_tag].view( 1, -1).expand(1, self.tagset_size)  #tensor([[-0.2473, -0.2473, -0.2473, -0.2473, -0.2473]], grad_fn=<ExpandBackward>)
                # the ith entry of trans_score is the score of transitioning to next_tag from i
                trans_score = self.transitions[next_tag].view(1, -1)  #(1,5)
                # score=  previous + now  + transition (previous -> now)                     before do log-sum-exp
                next_tag_var = forward_var + trans_score + emit_score #(1,5)
                # The forward variable for this tag is log-sum-exp of all the scores.
                alphas_t.append(log_sum_exp(next_tag_var).view(1))   #  append(tensor([-1.3660]))
            forward_var = torch.cat(alphas_t).view(1, -1)
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]     #（1，out)=(1,5)
        alpha = log_sum_exp(terminal_var)     #(1)
        return alpha



#    lstm 得到out=（b,m,out)--->(m,out)
    def _get_lstm_features(self, sentence):
        self.hidden = self.init_hidden()              #shape(2*2,1,2)= [num_layers(=2) * num_directions(=2), b(=1), h=(2)]
        embeds = self.word_embeds(sentence).view(len(sentence), 1, -1)  #shape=max_len*batch*embedding=(11,1,5)
        lstm_out, self.hidden = self.lstm(embeds, self.hidden)      #lstm_out=[m, b, num_directions(=2) * h(=2)] = (11,1,4)
        lstm_out = lstm_out.view(len(sentence), self.hidden_dim) #[m,num_directions(=2) * h(=2)] = (11,4)
        lstm_feats = self.hidden2tag(lstm_out)  #(11,4)*(4,out) =(11,out)=(11,5)=(m,out)
        return lstm_feats

#返回SCORE_realpath
    def _score_sentence(self, feats, tags):  #feats=(11,5)=(m,out) tags=(m)=11=tensor([0, 1, 1, 1, 2, 2, 2, 0, 1, 2, 2])
        # Gives the score of a provided tag sequence
        score = torch.zeros(1)
        tags = torch.cat([torch.tensor([self.tag_to_ix[START_TAG]], dtype=torch.long), tags])  #tensor([3, 0, 1, 1, 1, 2, 2, 2, 0, 1, 2, 2])
        for i, feat in enumerate(feats):
            #一次迭代表示一个单词位置,score =  score +P(now_real) + transition(now,next)
            score = score +  self.transitions[tags[i + 1], tags[i]] + feat[tags[i + 1]]
        score = score + self.transitions[self.tag_to_ix[STOP_TAG], tags[-1]]
        return score





# decode  for predict    得到预测的序列，以及预测序列的得分      emssion matric + transition <---> 最短路径问题 <--> viterbi
#https://zhuanlan.zhihu.com/p/79552594
    def _viterbi_decode(self, feats): #feats=(m,outs)=(11,5)
        backpointers = []

        # Initialize the viterbi variables in log space
        init_vvars = torch.full((1, self.tagset_size), -10000.)
        init_vvars[0][self.tag_to_ix[START_TAG]] = 0

        # forward_var at step i holds the viterbi variables for step i-1
        forward_var = init_vvars
        for feat in feats:
            bptrs_t = []  # holds the backpointers for this step
            viterbivars_t = []  # holds the viterbi variables for this step

            for next_tag in range(self.tagset_size):
                # next_tag_var[i] holds the viterbi variable for tag i at the
                # previous step, plus the score of transitioning
                # from tag i to next_tag.
                # We don't include the emission scores here because the max
                # does not depend on them (we add them in below)
                next_tag_var = forward_var + self.transitions[next_tag]
                best_tag_id = argmax(next_tag_var)
                bptrs_t.append(best_tag_id)
                viterbivars_t.append(next_tag_var[0][best_tag_id].view(1))
            # Now add in the emission scores, and assign forward_var to the set
            # of viterbi variables we just computed
            forward_var = (torch.cat(viterbivars_t) + feat).view(1, -1)
            backpointers.append(bptrs_t)

        # Transition to STOP_TAG
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        best_tag_id = argmax(terminal_var)
        path_score = terminal_var[0][best_tag_id]

        # Follow the back pointers to decode the best path.
        best_path = [best_tag_id]
        for bptrs_t in reversed(backpointers):
            best_tag_id = bptrs_t[best_tag_id]
            best_path.append(best_tag_id)
        # Pop off the start tag (we dont want to return that to the caller)
        start = best_path.pop()
        assert start == self.tag_to_ix[START_TAG]  # Sanity check
        best_path.reverse()
        return path_score, best_path


#for train           LOSS = -log(     P_realpath / (P_1+P_2+P_3+...)        )
#                         = -log(     e^S_realpath / (e^S_1+e^S_2+e^S_3+..)    )
#                         =  log(e^S_1  + e^S_2 + e^S_3 +...) - S_realpath
#                         =  forward_score - gold_score
    def neg_log_likelihood(self, sentence, tags):
        #lstm
        feats = self._get_lstm_features(sentence) #emission maxtrix ---> (11,5)=(m,out)
        #前向传播计算score                 forward_score = log(e^S_1  + e^S_2 + e^S_3 +...)
        forward_score = self._forward_alg(feats)  #forward_score=(1)
        #根据真实的tags计算score
        gold_score = self._score_sentence(feats, tags)
         #根据score返向传播学习
        return forward_score - gold_score



#for predict
    def forward(self, sentence):  # dont confuse this with _forward_alg above.
        # Get the emission scores from the BiLSTM
        lstm_feats = self._get_lstm_features(sentence)  #emission maxtrix  --->(11,5)=(m,out)

        # Find the best path, given the features.
        score, tag_seq = self._viterbi_decode(lstm_feats)
        return score, tag_seq

In [9]:

START_TAG = "<START>"
STOP_TAG = "<STOP>"
EMBEDDING_DIM = 5
HIDDEN_DIM = 4

# Make up some training data
training_data = [(
    "the wall street journal reported today that apple corporation made money".split(),
    "B I I I O O O B I O O".split()
), (
    "georgia tech is a university in georgia".split(),
    "B I O O O O B".split()
)]

word_to_ix = {}
for sentence, tags in training_data:
    for word in sentence:
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)

tag_to_ix = {"B": 0, "I": 1, "O": 2, START_TAG: 3, STOP_TAG: 4}
ix_to_tag = {v: k for k, v in tag_to_ix.items()}

print("word_to_ix", word_to_ix)
print("tag_to_ix", tag_to_ix)
print("ix_to_tag", ix_to_tag)

word_to_ix {'the': 0, 'wall': 1, 'street': 2, 'journal': 3, 'reported': 4, 'today': 5, 'that': 6, 'apple': 7, 'corporation': 8, 'made': 9, 'money': 10, 'georgia': 11, 'tech': 12, 'is': 13, 'a': 14, 'university': 15, 'in': 16}
tag_to_ix {'B': 0, 'I': 1, 'O': 2, '<START>': 3, '<STOP>': 4}
ix_to_tag {0: 'B', 1: 'I', 2: 'O', 3: '<START>', 4: '<STOP>'}


In [10]:
model = BiLSTM_CRF(len(word_to_ix), tag_to_ix, EMBEDDING_DIM, HIDDEN_DIM)
optimizer = optim.SGD(model.parameters(), lr=0.01, weight_decay=1e-4)

test = training_data[0][0]
# test = training_data[1][0]
prin
# Check predictions before training
with torch.no_grad():
    precheck_sent = prepare_sequence(test, word_to_ix)
    precheck_tags = torch.tensor([tag_to_ix[t] for t in training_data[0][1]], dtype=torch.long)
    before = model(precheck_sent)
    print("before  train   ", before)
    for ix in before[1]:
        print(ix_to_tag[ix], end=' ')

for epoch in range(300):
    for sentence, tags in training_data:
        model.zero_grad()

        sentence_in = prepare_sequence(sentence, word_to_ix)
        targets = torch.tensor([tag_to_ix[t] for t in tags], dtype=torch.long)

        # forward pass.
        loss = model.neg_log_likelihood(sentence_in, targets)

        loss.backward()
        optimizer.step()


test ['the', 'wall', 'street', 'journal', 'reported', 'today', 'that', 'apple', 'corporation', 'made', 'money']
before  train    (tensor(14.9282), [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2])
I B B B B B B B B B O 

In [11]:

# Check predictions after training
with torch.no_grad():
    precheck_sent = prepare_sequence(test, word_to_ix)
    after = model(precheck_sent)
    print("\nafter  train   ", after)
    for ix in after[1]:
        print(ix_to_tag[ix], end=' ')


after  train    (tensor(24.5888), [0, 1, 1, 1, 2, 2, 2, 0, 1, 2, 2])
B I I I O O O B I O O 